In [ ]:
#gpt-4o RAG

"""
Enhanced Multi-Knowledge Graph RAG System with Text Chunking
Handles large texts by processing them in chunks to avoid token limits
"""

import os
import re
import time
import json
import hashlib
from typing import List, Dict, Any, Optional, Tuple
import logging
from dataclasses import dataclass
from concurrent.futures import ThreadPoolExecutor, as_completed
import requests
import tiktoken

from dotenv import load_dotenv
from rdflib import Graph, RDFS, RDF, OWL, URIRef, Namespace, Literal
from rdflib.namespace import XSD, SKOS

# Configuration
INPUT_TEXT_FILE = "part_aa"
ONTOLOGY_PATH = "wiki.owl"
LOCATION_ONTOLOGY_PATH = "locations.owl"
OUTPUT_RAG_TTL = 'extracted_events_rag_with_multi_kg.ttl'
OUTPUT_RAG_OWL = 'extracted_events_rag_with_multi_kg.owl'
KG_CACHE_FILE = 'kg_cache.json'
LOCATION_CACHE_FILE = 'location_cache.json'
KG_ANALYSIS_REPORT = 'multi_kg_analysis_report.txt'

# Token limits
MAX_TOKENS_PER_REQUEST = 100000  # Conservative limit for GPT-4
CHUNK_OVERLAP = 200  # Characters to overlap between chunks

# Namespaces
EX = Namespace("http://example.org/")
STE = Namespace("http://www.example.org/ste#")
DBP = Namespace("http://dbpedia.org/ontology/")
LAC = Namespace("http://ontologia.fr/OTB/lac#")
WD = Namespace("http://www.wikidata.org/entity/")
YAGO = Namespace("http://yago-knowledge.org/resource/")
CN = Namespace("http://conceptnet.io/c/en/")
GEO = Namespace("http://www.w3.org/2003/01/geo/wgs84_pos#")
DBPR = Namespace("http://dbpedia.org/resource/")

# Logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Imports
try:
    from langchain_community.embeddings import HuggingFaceEmbeddings
    from langchain_community.vectorstores import FAISS
    from langchain_openai import ChatOpenAI
    from langchain.schema import HumanMessage
except ImportError as e:
    print(f"ImportError: {e}")
    print("pip install rdflib python-dotenv langchain langchain-openai langchain-community faiss-cpu sentence-transformers tiktoken requests")
    exit(1)

@dataclass
class LocationInfo:
    """Location information with coordinates"""
    name: str
    latitude: Optional[float] = None
    longitude: Optional[float] = None
    country: Optional[str] = None
    region: Optional[str] = None
    source: str = "extracted"
    confidence: float = 1.0
    uri: Optional[str] = None

@dataclass
class EnhancedKnowledgeFact:
    """Enhanced knowledge fact with metadata"""
    subject: str
    predicate: str
    object: str
    source: str
    confidence: float = 1.0
    context: Optional[str] = None
    temporal: Optional[str] = None
    spatial: Optional[str] = None
    evidence_score: float = 1.0
    source_uri: Optional[str] = None

class LocationExtractor:
    """Extracts and enriches location information"""
    
    def __init__(self, ontology_path: str = LOCATION_ONTOLOGY_PATH):
        self.ontology_path = ontology_path
        self.location_graph = None
        self.location_cache = self._load_location_cache()
        self.load_location_ontology()
        
    def _load_location_cache(self) -> Dict:
        """Load location cache"""
        if os.path.exists(LOCATION_CACHE_FILE):
            try:
                with open(LOCATION_CACHE_FILE, 'r', encoding='utf-8') as f:
                    return json.load(f)
            except Exception as e:
                logger.warning(f"Could not load location cache: {e}")
        return {}
    
    def _save_location_cache(self):
        """Save location cache"""
        try:
            with open(LOCATION_CACHE_FILE, 'w', encoding='utf-8') as f:
                json.dump(self.location_cache, f, indent=2, ensure_ascii=False)
        except Exception as e:
            logger.warning(f"Could not save location cache: {e}")
    
    def load_location_ontology(self):
        """Load locations.owl ontology"""
        try:
            if os.path.exists(self.ontology_path):
                self.location_graph = Graph()
                self.location_graph.parse(self.ontology_path, format="xml")
                logger.info(f"Loaded location ontology from {self.ontology_path}")
            else:
                logger.warning(f"Location ontology not found at {self.ontology_path}")
                self.location_graph = None
        except Exception as e:
            logger.error(f"Error loading location ontology: {e}")
            self.location_graph = None
    
    def extract_locations_from_text(self, text: str) -> List[str]:
        """Extract potential location names from text"""
        # Pattern for location names (capitalized words, possibly with common location indicators)
        location_patterns = [
            r'\b[A-Z][a-zA-Z]+(?:\s+[A-Z][a-zA-Z]+)*(?:\s+(?:City|County|State|Province|Country|Region|Island|Bay|Sea|Ocean|River|Mountain|Valley|Desert))\b',
            r'\b(?:Mount|Lake|River|Cape|Fort|Port|Saint|St\.)\s+[A-Z][a-zA-Z]+(?:\s+[A-Z][a-zA-Z]+)*\b',
            r'\b[A-Z][a-zA-Z]+(?:\s+[A-Z][a-zA-Z]+)*(?=\s+(?:in|near|at|from|to))\b',
            r'\b[A-Z][a-zA-Z]{2,}(?:\s+[A-Z][a-zA-Z]{2,})*\b'  # General capitalized names
        ]
        
        locations = []
        for pattern in location_patterns:
            matches = re.findall(pattern, text)
            locations.extend(matches)
        
        # Filter and clean
        location_stopwords = {
            'The', 'This', 'That', 'These', 'Those', 'And', 'But', 'Or', 'So', 'If', 
            'When', 'Where', 'Who', 'What', 'How', 'Why', 'All', 'Some', 'Many', 'Most',
            'First', 'Second', 'Third', 'Last', 'Next', 'Before', 'After', 'During',
            'January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 
            'September', 'October', 'November', 'December', 'Monday', 'Tuesday', 
            'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'
        }
        
        filtered_locations = []
        for loc in locations:
            loc = loc.strip()
            if (loc not in location_stopwords and len(loc) > 2 and 
                not loc.isdigit() and not re.match(r'^\d+', loc)):
                filtered_locations.append(loc)
        
        return list(set(filtered_locations))
    
    def get_location_from_ontology(self, location_name: str) -> Optional[LocationInfo]:
        """Get location info from local ontology"""
        if not self.location_graph:
            return None
            
        try:
            # Query the ontology for location information
            query = f"""
            SELECT DISTINCT ?location ?lat ?long ?country ?region WHERE {{
                ?location rdfs:label ?label .
                FILTER(regex(?label, "{location_name}", "i"))
                OPTIONAL {{ ?location geo:lat ?lat }}
                OPTIONAL {{ ?location geo:long ?long }}
                OPTIONAL {{ ?location dbp:country ?country }}
                OPTIONAL {{ ?location dbp:region ?region }}
            }}
            """
            
            results = self.location_graph.query(query)
            for row in results:
                return LocationInfo(
                    name=location_name,
                    latitude=float(row.lat) if row.lat else None,
                    longitude=float(row.long) if row.long else None,
                    country=str(row.country) if row.country else None,
                    region=str(row.region) if row.region else None,
                    source="local_ontology",
                    uri=str(row.location) if row.location else None
                )
        except Exception as e:
            logger.debug(f"Ontology query failed for {location_name}: {e}")
        
        return None
    
    def get_location_from_dbpedia(self, location_name: str) -> Optional[LocationInfo]:
        """Get location coordinates from DBpedia"""
        try:
            time.sleep(0.5)  # Rate limiting
            
            # Try to find the DBpedia resource
            entity_uri = f"http://dbpedia.org/resource/{location_name.replace(' ', '_')}"
            
            sparql_query = f"""
            SELECT DISTINCT ?lat ?long ?country ?region WHERE {{
                <{entity_uri}> geo:lat ?lat ;
                               geo:long ?long .
                OPTIONAL {{ <{entity_uri}> dbo:country ?country }}
                OPTIONAL {{ <{entity_uri}> dbo:region ?region }}
            }}
            """
            
            params = {'query': sparql_query, 'format': 'json'}
            response = requests.get("https://dbpedia.org/sparql", params=params, timeout=10)
            
            if response.status_code == 200:
                data = response.json()
                bindings = data.get('results', {}).get('bindings', [])
                
                if bindings:
                    binding = bindings[0]
                    return LocationInfo(
                        name=location_name,
                        latitude=float(binding.get('lat', {}).get('value', 0)),
                        longitude=float(binding.get('long', {}).get('value', 0)),
                        country=binding.get('country', {}).get('value', ''),
                        region=binding.get('region', {}).get('value', ''),
                        source="dbpedia",
                        uri=entity_uri
                    )
                    
        except Exception as e:
            logger.debug(f"DBpedia location query failed for {location_name}: {e}")
        
        return None
    
    def get_location_from_wikidata(self, location_name: str) -> Optional[LocationInfo]:
        """Get location coordinates from Wikidata"""
        try:
            time.sleep(0.5)  # Rate limiting
            
            sparql_query = f"""
            SELECT DISTINCT ?item ?itemLabel ?coord ?country ?countryLabel WHERE {{
              ?item rdfs:label "{location_name}"@en .
              ?item wdt:P625 ?coord .
              OPTIONAL {{ ?item wdt:P17 ?country }}
              SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
            }}
            LIMIT 1
            """
            
            params = {'query': sparql_query, 'format': 'json'}
            response = requests.get("https://query.wikidata.org/sparql", params=params, timeout=10)
            
            if response.status_code == 200:
                data = response.json()
                bindings = data.get('results', {}).get('bindings', [])
                
                if bindings:
                    binding = bindings[0]
                    coord_str = binding.get('coord', {}).get('value', '')
                    
                    # Parse coordinate string (format: "Point(longitude latitude)")
                    coord_match = re.search(r'Point\(([+-]?\d*\.?\d+)\s+([+-]?\d*\.?\d+)\)', coord_str)
                    if coord_match:
                        longitude = float(coord_match.group(1))
                        latitude = float(coord_match.group(2))
                        
                        return LocationInfo(
                            name=location_name,
                            latitude=latitude,
                            longitude=longitude,
                            country=binding.get('countryLabel', {}).get('value', ''),
                            source="wikidata",
                            uri=binding.get('item', {}).get('value', '')
                        )
                        
        except Exception as e:
            logger.debug(f"Wikidata location query failed for {location_name}: {e}")
        
        return None
    
    def enrich_location(self, location_name: str) -> Optional[LocationInfo]:
        """Get enriched location information with coordinates"""
        # Check cache first
        if location_name in self.location_cache:
            cached = self.location_cache[location_name]
            return LocationInfo(**cached) if cached else None
        
        # Try different sources in order of preference
        location_info = None
        
        # 1. Try local ontology first
        location_info = self.get_location_from_ontology(location_name)
        
        # 2. Try Wikidata
        if not location_info:
            location_info = self.get_location_from_wikidata(location_name)
        
        # 3. Try DBpedia
        if not location_info:
            location_info = self.get_location_from_dbpedia(location_name)
        
        # Cache the result (even if None)
        if location_info:
            self.location_cache[location_name] = {
                'name': location_info.name,
                'latitude': location_info.latitude,
                'longitude': location_info.longitude,
                'country': location_info.country,
                'region': location_info.region,
                'source': location_info.source,
                'confidence': location_info.confidence,
                'uri': location_info.uri
            }
        else:
            self.location_cache[location_name] = None
        
        self._save_location_cache()
        return location_info

class TextChunker:
    """Handles text chunking to manage token limits"""
    
    def __init__(self, model_name: str = "gpt-4"):
        self.tokenizer = tiktoken.encoding_for_model(model_name)
    
    def count_tokens(self, text: str) -> int:
        """Count tokens in text"""
        return len(self.tokenizer.encode(text))
    
    def chunk_text_by_tokens(self, text: str, max_tokens: int = 15000, overlap_tokens: int = 200) -> List[str]:
        """Chunk text by token count with overlap"""
        tokens = self.tokenizer.encode(text)
        chunks = []
        
        start = 0
        while start < len(tokens):
            end = min(start + max_tokens, len(tokens))
            chunk_tokens = tokens[start:end]
            chunk_text = self.tokenizer.decode(chunk_tokens)
            chunks.append(chunk_text)
            
            if end >= len(tokens):
                break
            
            # Move start back by overlap amount
            start = end - overlap_tokens
        
        return chunks
    
    def chunk_text_by_sentences(self, text: str, max_tokens: int = 15000) -> List[str]:
        """Chunk text by sentences to maintain coherence"""
        sentences = re.split(r'[.!?]+', text)
        chunks = []
        current_chunk = ""
        
        for sentence in sentences:
            sentence = sentence.strip()
            if not sentence:
                continue
                
            test_chunk = current_chunk + " " + sentence if current_chunk else sentence
            
            if self.count_tokens(test_chunk) > max_tokens and current_chunk:
                chunks.append(current_chunk.strip())
                current_chunk = sentence
            else:
                current_chunk = test_chunk
        
        if current_chunk.strip():
            chunks.append(current_chunk.strip())
        
        return chunks

class BaseKGConnector:
    """Base class for knowledge graph connectors"""
    
    def __init__(self, name: str, base_url: str, rate_limit: float = 1.0):
        self.name = name
        self.base_url = base_url
        self.rate_limit = rate_limit
        self.last_request_time = 0
        self.request_count = 0
        self.success_count = 0
        
    def _rate_limit_wait(self):
        """Enforce rate limiting"""
        current_time = time.time()
        time_since_last = current_time - self.last_request_time
        if time_since_last < self.rate_limit:
            time.sleep(self.rate_limit - time_since_last)
        self.last_request_time = time.time()
        self.request_count += 1
    
    def get_stats(self) -> Dict[str, Any]:
        """Get connector statistics"""
        return {
            'name': self.name,
            'requests': self.request_count,
            'successes': self.success_count,
            'success_rate': self.success_count / max(1, self.request_count)
        }
    
    def retrieve_facts(self, entity: str, limit: int = 50) -> List[EnhancedKnowledgeFact]:
        """Abstract method to retrieve facts"""
        raise NotImplementedError

class EnhancedWikidataConnector(BaseKGConnector):
    """Wikidata connector"""
    
    def __init__(self):
        super().__init__("Wikidata", "https://query.wikidata.org/sparql", 1.0)
        
    def retrieve_facts(self, entity: str, limit: int = 50) -> List[EnhancedKnowledgeFact]:
        """Retrieve facts from Wikidata"""
        try:
            self._rate_limit_wait()
            
            sparql_query = f"""
            SELECT DISTINCT ?subject ?subjectLabel ?predicate ?predicateLabel ?object ?objectLabel WHERE {{
              {{
                ?subject ?label "{entity}"@en .
              }} UNION {{
                ?subject rdfs:label "{entity}"@en .
              }}
              
              ?subject ?predicate ?object .
              FILTER(?predicate != wdt:P31 && ?predicate != wdt:P279)
              
              SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
            }}
            LIMIT {limit}
            """
            
            params = {'query': sparql_query, 'format': 'json'}
            response = requests.get(self.base_url, params=params, timeout=15)
            
            if response.status_code == 200:
                data = response.json()
                facts = []
                
                for binding in data.get('results', {}).get('bindings', []):
                    fact = EnhancedKnowledgeFact(
                        subject=binding.get('subjectLabel', {}).get('value', entity),
                        predicate=binding.get('predicateLabel', {}).get('value', 'related_to'),
                        object=binding.get('objectLabel', {}).get('value', ''),
                        source=self.name,
                        confidence=0.9,
                        source_uri=binding.get('subject', {}).get('value')
                    )
                    facts.append(fact)
                
                self.success_count += 1
                logger.info(f"Retrieved {len(facts)} facts from Wikidata for '{entity}'")
                return facts
                
        except Exception as e:
            logger.error(f"Wikidata query failed for '{entity}': {e}")
        
        return []

class EnhancedDBpediaConnector(BaseKGConnector):
    """DBpedia connector"""
    
    def __init__(self):
        super().__init__("DBpedia", "https://dbpedia.org/sparql", 1.0)
        
    def retrieve_facts(self, entity: str, limit: int = 50) -> List[EnhancedKnowledgeFact]:
        """Retrieve facts from DBpedia"""
        try:
            self._rate_limit_wait()
            
            entity_uri = f"http://dbpedia.org/resource/{entity.replace(' ', '_')}"
            
            sparql_query = f"""
            SELECT DISTINCT ?predicate ?object WHERE {{
              <{entity_uri}> ?predicate ?object .
              FILTER(LANG(?object) = "en" || !isLiteral(?object))
              FILTER(!isBlank(?object))
            }}
            LIMIT {limit}
            """
            
            params = {'query': sparql_query, 'format': 'json'}
            response = requests.get(self.base_url, params=params, timeout=15)
            
            if response.status_code == 200:
                data = response.json()
                facts = []
                
                for binding in data.get('results', {}).get('bindings', []):
                    predicate = binding.get('predicate', {}).get('value', '')
                    obj = binding.get('object', {}).get('value', '')
                    
                    predicate_name = predicate.split('/')[-1].replace('_', ' ')
                    
                    fact = EnhancedKnowledgeFact(
                        subject=entity,
                        predicate=predicate_name,
                        object=obj,
                        source=self.name,
                        confidence=0.85,
                        source_uri=entity_uri
                    )
                    facts.append(fact)
                
                self.success_count += 1
                logger.info(f"Retrieved {len(facts)} facts from DBpedia for '{entity}'")
                return facts
                
        except Exception as e:
            logger.error(f"DBpedia query failed for '{entity}': {e}")
        
        return []

class EnhancedConceptNetConnector(BaseKGConnector):
    """ConceptNet connector"""
    
    def __init__(self):
        super().__init__("ConceptNet", "http://api.conceptnet.io", 0.5)
        
    def retrieve_facts(self, entity: str, limit: int = 50) -> List[EnhancedKnowledgeFact]:
        """Retrieve facts from ConceptNet"""
        try:
            self._rate_limit_wait()
            
            concept = f"/c/en/{entity.lower().replace(' ', '_')}"
            url = f"{self.base_url}{concept}?limit={limit}"
            
            response = requests.get(url, timeout=10)
            
            if response.status_code == 200:
                data = response.json()
                facts = []
                
                for edge in data.get('edges', []):
                    start = edge.get('start', {})
                    end = edge.get('end', {})
                    relation = edge.get('rel', {})
                    weight = edge.get('weight', 1.0)
                    
                    start_label = start.get('label', '').replace('/c/en/', '').replace('_', ' ')
                    end_label = end.get('label', '').replace('/c/en/', '').replace('_', ' ')
                    rel_label = relation.get('label', 'related_to')
                    
                    fact = EnhancedKnowledgeFact(
                        subject=start_label,
                        predicate=rel_label,
                        object=end_label,
                        source=self.name,
                        confidence=min(weight, 1.0)
                    )
                    facts.append(fact)
                
                self.success_count += 1
                logger.info(f"Retrieved {len(facts)} facts from ConceptNet for '{entity}'")
                return facts
                
        except Exception as e:
            logger.error(f"ConceptNet query failed for '{entity}': {e}")
        
        return []

class MultiKGCache:
    """Caching system for knowledge graph facts"""
    
    def __init__(self, cache_file: str = KG_CACHE_FILE):
        self.cache_file = cache_file
        self.cache = self._load_cache()
        
    def _load_cache(self) -> Dict:
        """Load cache from file"""
        if os.path.exists(self.cache_file):
            try:
                with open(self.cache_file, 'r', encoding='utf-8') as f:
                    return json.load(f)
            except Exception as e:
                logger.warning(f"Could not load cache: {e}")
        return {}
    
    def _save_cache(self):
        """Save cache to file"""
        try:
            with open(self.cache_file, 'w', encoding='utf-8') as f:
                json.dump(self.cache, f, indent=2, ensure_ascii=False)
        except Exception as e:
            logger.warning(f"Could not save cache: {e}")
    
    def get_cache_key(self, source: str, entity: str) -> str:
        """Generate cache key"""
        return f"{source}:{hashlib.md5(entity.encode()).hexdigest()}"
    
    def get(self, source: str, entity: str) -> Optional[List[Dict]]:
        """Get cached facts"""
        key = self.get_cache_key(source, entity)
        return self.cache.get(key)
    
    def set(self, source: str, entity: str, facts: List[EnhancedKnowledgeFact]):
        """Cache facts"""
        key = self.get_cache_key(source, entity)
        serializable_facts = []
        for fact in facts:
            serializable_facts.append({
                'subject': fact.subject,
                'predicate': fact.predicate,
                'object': fact.object,
                'source': fact.source,
                'confidence': fact.confidence,
                'context': fact.context,
                'temporal': fact.temporal,
                'spatial': fact.spatial,
                'evidence_score': fact.evidence_score,
                'source_uri': fact.source_uri
            })
        self.cache[key] = serializable_facts
        self._save_cache()

class EnhancedMultiKGRAGSystem:
    """Multi-Knowledge Graph RAG system with chunking and location extraction"""
    
    def __init__(self):
        self.connectors = {
            'wikidata': EnhancedWikidataConnector(),
            'dbpedia': EnhancedDBpediaConnector(),
            'conceptnet': EnhancedConceptNetConnector()
        }
        self.cache = MultiKGCache()
        self.chunker = TextChunker()
        self.location_extractor = LocationExtractor()
        self.stats = {
            'queries_processed': 0,
            'entities_extracted': 0,
            'facts_retrieved': 0,
            'cache_hits': 0,
            'chunks_processed': 0,
            'locations_found': 0,
            'locations_with_coordinates': 0
        }
        
    def extract_entities_advanced(self, text: str) -> List[str]:
        """Extract entities from text"""
        entities = []
        
        # Pattern for capitalized names
        pattern = r'\b[A-Z][a-zA-Z]+(?:\s+[A-Z][a-zA-Z]+)*\b'
        matches = re.findall(pattern, text)
        entities.extend(matches)
        
        # Filter out common stop words
        stop_words = {
            'The', 'This', 'That', 'These', 'Those', 'And', 'But', 'Or', 'So', 'If', 'When', 'Where',
            'Who', 'What', 'How', 'Why', 'All', 'Some', 'Many', 'Few', 'Most', 'Each', 'Every',
            'First', 'Second', 'Third', 'Last', 'Next', 'Previous', 'Before', 'After', 'During'
        }
        
        filtered_entities = []
        for entity in entities:
            entity = entity.strip()
            if (entity not in stop_words and len(entity) > 2 and not entity.isdigit()):
                filtered_entities.append(entity)
        
        # Remove duplicates
        seen = set()
        unique_entities = []
        for entity in filtered_entities:
            if entity.lower() not in seen:
                seen.add(entity.lower())
                unique_entities.append(entity)
        
        return unique_entities[:15]  # Limit to prevent too many API calls
    
    def retrieve_kg_facts_enhanced(self, entities: List[str]) -> Dict[str, List[EnhancedKnowledgeFact]]:
        """Retrieve facts from knowledge graphs"""
        all_facts = {}
        cache_hits = 0
        
        with ThreadPoolExecutor(max_workers=3) as executor:
            futures = {}
            
            for entity in entities:
                for source_name, connector in self.connectors.items():
                    # Check cache first
                    cached_facts = self.cache.get(source_name, entity)
                    if cached_facts:
                        cache_hits += 1
                        if entity not in all_facts:
                            all_facts[entity] = []
                        for fact_data in cached_facts:
                            fact = EnhancedKnowledgeFact(**fact_data)
                            all_facts[entity].append(fact)
                    else:
                        future = executor.submit(connector.retrieve_facts, entity, 3)
                        futures[future] = (entity, source_name)
            
            # Collect results
            for future in as_completed(futures, timeout=30):
                entity, source_name = futures[future]
                try:
                    facts = future.result()
                    if facts:
                        self.cache.set(source_name, entity, facts)
                        
                        if entity not in all_facts:
                            all_facts[entity] = []
                        all_facts[entity].extend(facts)
                        
                        self.stats['facts_retrieved'] += len(facts)
                    
                except Exception as e:
                    logger.error(f"Failed to retrieve facts from {source_name} for {entity}: {e}")
        
        self.stats['cache_hits'] += cache_hits
        return all_facts
    
    def format_kg_context_enhanced(self, kg_facts: Dict[str, List[EnhancedKnowledgeFact]]) -> str:
        """Format KG facts into context string"""
        context_parts = []
        
        for entity, facts in kg_facts.items():
            if facts:
                # Sort facts by confidence
                sorted_facts = sorted(facts, key=lambda f: f.confidence, reverse=True)
                
                context_parts.append(f"\n=== Knowledge about {entity} ===")
                
                # Group facts by source
                by_source = {}
                for fact in sorted_facts[:3]:  # Limit facts per entity
                    if fact.source not in by_source:
                        by_source[fact.source] = []
                    by_source[fact.source].append(fact)
                
                for source, source_facts in by_source.items():
                    context_parts.append(f"\nFrom {source}:")
                    for fact in source_facts[:2]:  # Limit facts per source
                        fact_str = f"- {fact.subject} {fact.predicate} {fact.object}"
                        if fact.confidence < 0.8:
                            fact_str += f" (confidence: {fact.confidence:.2f})"
                        context_parts.append(fact_str)
        
        return "\n".join(context_parts)
    
    def process_chunk(self, chunk: str, chunk_num: int, llm) -> str:
        """Process a single chunk of text with location extraction"""
        logger.info(f"Processing chunk {chunk_num} ({len(chunk)} chars)")
        
        # Extract entities from this chunk
        entities = self.extract_entities_advanced(chunk)
        
        # Extract locations from this chunk
        locations = self.location_extractor.extract_locations_from_text(chunk)
        logger.info(f"Found potential locations in chunk {chunk_num}: {locations}")
        
        # Enrich locations with coordinates
        enriched_locations = {}
        for location_name in locations[:10]:  # Limit to prevent too many API calls
            location_info = self.location_extractor.enrich_location(location_name)
            if location_info:
                enriched_locations[location_name] = location_info
                self.stats['locations_found'] += 1
                if location_info.latitude and location_info.longitude:
                    self.stats['locations_with_coordinates'] += 1
        
        if not entities and not enriched_locations:
            logger.info(f"No entities or locations found in chunk {chunk_num}")
            return ""
        
        logger.info(f"Found entities in chunk {chunk_num}: {entities[:5]}...")
        logger.info(f"Enriched {len(enriched_locations)} locations with coordinates")
        
        # Get KG facts for entities in this chunk
        kg_facts = self.retrieve_kg_facts_enhanced(entities)
        kg_context = self.format_kg_context_enhanced(kg_facts)
        
        # Format location context
        location_context = self.format_location_context(enriched_locations)
        
        # Create prompt for this chunk
        enhanced_prompt = f"""You are extracting historical events from this specific text chunk. Use the knowledge graph facts and location information to enhance your extraction.

TEXT CHUNK {chunk_num}:
{chunk}

KNOWLEDGE GRAPH FACTS FOR ENTITIES IN THIS CHUNK:
{kg_context}

LOCATION INFORMATION WITH COORDINATES:
{location_context}

TASK: Extract ONLY the events that are actually mentioned in this text chunk. Use the KG facts and location coordinates to enhance details but stay faithful to what's actually in the text.

Requirements:
1. Extract ONLY events mentioned in this text chunk
2. Use proper RDF/Turtle format
3. Include these properties for each event:
   - ste:hasType (description of event)
   - ste:hasAgent (who caused/led the event)
   - ste:hasTime (when it happened)
   - ste:hasLocation (location name from text)
   - ste:hasLatitude (latitude coordinate if available)
   - ste:hasLongitude (longitude coordinate if available)
   - ste:hasCountry (country if available)
   - ste:hasResult (outcome/consequence)

Output format (do not include prefixes, they will be added later):
```turtle
ste:Event{chunk_num}_1 a ste:Event, dbp:SpecificEventType ;
    ste:hasType "specific description from text" ;
    ste:hasAgent "specific person from text" ;
    ste:hasTime "specific date from text" ;
    ste:hasLocation "specific location from text" ;
    ste:hasLatitude "latitude_value"^^xsd:double ;
    ste:hasLongitude "longitude_value"^^xsd:double ;
    ste:hasCountry "country_name" ;
    ste:hasResult "specific outcome from text" .

ste:Location{chunk_num}_1 a ste:Location ;
    rdfs:label "location_name" ;
    geo:lat "latitude_value"^^xsd:double ;
    geo:long "longitude_value"^^xsd:double ;
    ste:hasCountry "country_name" .
```

IMPORTANT: 
- Only extract events that are explicitly mentioned in this chunk
- Use the exact coordinates provided in the location information
- Only include coordinate properties if coordinates are available
- If no clear events are found, return empty
- Create separate location entities for places mentioned in events
"""
        
        try:
            response = llm.invoke([HumanMessage(content=enhanced_prompt)])
            turtle_output = self.clean_turtle(response.content)
            self.stats['chunks_processed'] += 1
            return turtle_output
        except Exception as e:
            logger.error(f"Error processing chunk {chunk_num}: {e}")
            return ""
    
    def format_location_context(self, enriched_locations: Dict[str, LocationInfo]) -> str:
        """Format location information into context string"""
        if not enriched_locations:
            return "No location coordinates available."
        
        context_parts = ["\n=== Location Information ==="]
        
        for location_name, location_info in enriched_locations.items():
            context_parts.append(f"\n{location_name}:")
            context_parts.append(f"  - Source: {location_info.source}")
            
            if location_info.latitude and location_info.longitude:
                context_parts.append(f"  - Coordinates: {location_info.latitude}, {location_info.longitude}")
            else:
                context_parts.append("  - Coordinates: Not available")
            
            if location_info.country:
                context_parts.append(f"  - Country: {location_info.country}")
            
            if location_info.region:
                context_parts.append(f"  - Region: {location_info.region}")
            
            if location_info.uri:
                context_parts.append(f"  - URI: {location_info.uri}")
        
        return "\n".join(context_parts)
    
    def clean_turtle(self, raw_output: str) -> str:
        """Clean turtle output"""
        m = re.search(r"```(?:turtle)?\s*(.*?)```", raw_output, re.DOTALL | re.IGNORECASE)
        if m:
            return m.group(1).strip()
        
        lines = raw_output.strip().split('\n')
        turtle_lines = []
        for line in lines:
            stripped = line.strip()
            if (stripped.startswith('@') or stripped.startswith('<') or 
                stripped.startswith(':') or stripped.startswith('_') or 
                stripped.startswith('a ') or ':' in stripped or stripped == ''):
                turtle_lines.append(line)
        
        return "\n".join(turtle_lines).strip() if turtle_lines else raw_output.strip()

def load_api_key():
    """Load OpenAI API key"""
    load_dotenv()
    api_key = os.getenv("OPENAI_API_KEY")
    if not api_key:
        print("Error: OPENAI_API_KEY not found")
        return None
    print("OpenAI API Key loaded successfully.")
    return api_key

def load_text_from_file(filepath: str) -> str:
    """Load text from file"""
    if not os.path.isfile(filepath):
        print(f"File not found: {filepath}")
        return ""
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            text = f.read().strip()
        print(f"Loaded text from {filepath}")
        return text
    except Exception as e:
        print(f"Error reading file {filepath}: {e}")
        return ""

def initialize_llm(api_key: str):
    """Initialize LLM"""
    if not api_key:
        return None
    try:
        llm = ChatOpenAI(model_name="gpt-4o", temperature=0, openai_api_key=api_key)
        print("LLM initialized successfully.")
        return llm
    except Exception as e:
        print(f"Error initializing LLM: {e}")
        return None

def main():
    """Main function with chunking support"""
    print("🚀 Starting Multi-Knowledge Graph RAG System with Chunking")
    
    # Load API key
    api_key = load_api_key()
    if not api_key:
        return
    
    # Load YOUR actual text
    domain_text = load_text_from_file(INPUT_TEXT_FILE)
    if not domain_text:
        print("⚠️  No input file found, using sample text")
        domain_text = """The Battle of Salamis was a decisive naval battle in 480 BC. 
        Themistocles led the Greek fleet to victory over the Persians commanded by Xerxes. 
        This victory established Greek naval supremacy in the Aegean Sea."""
    else:
        print(f"📄 Using YOUR text from {INPUT_TEXT_FILE}")
        print(f"📝 Text length: {len(domain_text)} characters")
    
    # Initialize systems
    multi_kg_system = EnhancedMultiKGRAGSystem()
    llm = initialize_llm(api_key)
    
    if not llm:
        return
    
    # Check if text needs chunking
    token_count = multi_kg_system.chunker.count_tokens(domain_text)
    print(f"🔢 Total tokens in text: {token_count:,}")
    
    if token_count > 15000:  # Conservative chunking threshold
        print("📊 Text is large, chunking into smaller pieces...")
        chunks = multi_kg_system.chunker.chunk_text_by_sentences(domain_text, max_tokens=15000)
        print(f"📄 Created {len(chunks)} chunks")
    else:
        print("📄 Text is small enough to process as single chunk")
        chunks = [domain_text]
    
    # Process each chunk
    all_turtle_outputs = []
    all_entities = set()
    
    for i, chunk in enumerate(chunks, 1):
        print(f"\n🔄 Processing chunk {i}/{len(chunks)}...")
        
        turtle_output = multi_kg_system.process_chunk(chunk, i, llm)
        if turtle_output:
            all_turtle_outputs.append(turtle_output)
            
        # Extract entities for statistics
        chunk_entities = multi_kg_system.extract_entities_advanced(chunk)
        all_entities.update(chunk_entities)
        
        # Small delay to be respectful to APIs
        if i < len(chunks):
            time.sleep(1)
    
    # Combine all outputs
    if all_turtle_outputs:
        # Add prefixes
        prefixes = """@prefix ste: <http://www.example.org/ste#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix dbp: <http://dbpedia.org/ontology/> .
@prefix geo: <http://www.w3.org/2003/01/geo/wgs84_pos#> .
@prefix dbpr: <http://dbpedia.org/resource/> .

"""
        
        final_output = prefixes + "\n\n".join(all_turtle_outputs)
        
        # Save output
        with open(OUTPUT_RAG_TTL, 'w', encoding='utf-8') as f:
            f.write(final_output)
        
        print(f"\n✅ Saved enhanced RDF to {OUTPUT_RAG_TTL}")
        print(f"📊 Processing Statistics:")
        print(f"   - Total chunks processed: {len(chunks)}")
        print(f"   - Successful chunks: {len(all_turtle_outputs)}")
        print(f"   - Unique entities found: {len(all_entities)}")
        print(f"   - Total KG facts retrieved: {multi_kg_system.stats['facts_retrieved']}")
        print(f"   - Cache hits: {multi_kg_system.stats['cache_hits']}")
        print(f"   - Locations found: {multi_kg_system.stats['locations_found']}")
        print(f"   - Locations with coordinates: {multi_kg_system.stats['locations_with_coordinates']}")
        
        # Show connector statistics
        print(f"\n🔗 Knowledge Graph Connector Statistics:")
        for name, connector in multi_kg_system.connectors.items():
            stats = connector.get_stats()
            print(f"   - {stats['name']}: {stats['successes']}/{stats['requests']} requests ({stats['success_rate']:.1%} success)")
        
        # Show location extraction statistics
        if multi_kg_system.location_extractor.location_cache:
            successful_locations = sum(1 for v in multi_kg_system.location_extractor.location_cache.values() if v is not None)
            total_locations = len(multi_kg_system.location_extractor.location_cache)
            print(f"   - Location enrichment: {successful_locations}/{total_locations} locations enriched ({successful_locations/total_locations:.1%} success)")
        
        # Show sample of output
        print(f"\n📝 Sample of generated RDF:")
        print("="*60)
        print(final_output[:1000] + "..." if len(final_output) > 1000 else final_output)
        print("="*60)
        
    else:
        print("❌ No events were extracted from any chunks")
    
    print(f"\n🎉 Process complete! Check {OUTPUT_RAG_TTL} for results based on YOUR input text.")

if __name__ == '__main__':
    main()